In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

def mask_my_frame(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = image.shape[:2]
    polygons = np.array([[(20, 50), (20, height-50), (width-20, height-50), (width-20, 50)]])
    mask = np.zeros_like(gray)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_or(image, image, mask=mask)
    # masked_image = cv2.bitwise_and(image, mask)
    return masked_image


def stabilize(frame):
    global frame_queue, frame_queue_indexes, previous_gray, previous_keypoints, path, smoothed_path, frame_transforms_smoothed, frame_height, frame_width

    if frame is None:
        return

    if not frame_queue:	
        previous_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        previous_gray = clahe.apply(previous_gray)
        # cv2.imshow("gray", previous_gray)
        previous_keypoints = cv2.goodFeaturesToTrack(previous_gray, maxCorners=200, qualityLevel=0.05, minDistance=30.0, blockSize=3 , mask=None, useHarrisDetector=False, k=0.04) #track features using GFTT
        frame_height, frame_width = frame.shape[:2]
        frame_queue.append(frame)
        frame_queue_indexes.append(0)
        previous_gray = previous_gray[:]

    elif frame_queue_indexes[-1] <= 24:
        frame_queue.append(frame)
        frame_queue_indexes.append(frame_queue_indexes[-1]+1)
        frame_transform = generate_transformations()
        if frame_queue_indexes[-1] == 24:
            for i in range(3):
                smoothed_path[:,i] = box_filter_convolve((path[:,i]), window_size=25)
            deviation = smoothed_path - path
            frame_transforms_smoothed = frame_transform + deviation

    else:
        frame_queue.append(frame)
        frame_queue_indexes.append(frame_queue_indexes[-1]+1)
        # print(frame_queue_indexes)
        frame_transform = generate_transformations()

        for i in range(3):
            smoothed_path[:,i] = box_filter_convolve((path[:,i]), window_size=25)

        deviation = smoothed_path - path
        frame_transforms_smoothed = frame_transform + deviation

        return apply_transformations()

    return frame 


#Moving average filter used for smoothing the data 
def box_filter_convolve(path, window_size):
    box_filter = np.ones(25)/25
    path_padded = np.pad(path, (window_size, window_size), 'mean')
    path_smoothed = np.convolve(path_padded, box_filter, mode='same')
    path_smoothed = path_smoothed[window_size:-window_size]
    # assert path.shape == path_smoothed.shape
    return path_smoothed


# Estimating Rigid Transformations
def generate_transformations():

    global frame_queue, previous_gray, previous_keypoints, path, smoothed_path
    frame_gray = cv2.cvtColor(frame_queue[-1], cv2.COLOR_BGR2GRAY)
    frame_gray = clahe.apply(frame_gray)

    #optical flow using Lucas-Kanade differential method
    curr_kps, status, error = cv2.calcOpticalFlowPyrLK(previous_gray, frame_gray, previous_keypoints, None)
    valid_curr_kps = curr_kps[status==1] 
    valid_previous_keypoints = previous_keypoints[status==1] 
    # Rigid Transformation
    transformation = cv2.estimateRigidTransform(valid_previous_keypoints,valid_curr_kps, False)

    if not(transformation is None):
        # pevious_2_current translation in x direction
        dx = transformation[0, 2]
        # pevious_2_current translation in y direction
        dy = transformation[1, 2]
        # pevious_2_current rotation in angle
        da = np.arctan2(transformation[1, 0], transformation[0, 0])
    else:
        dx = dy = da = 0

    transforms.append([dx, dy, da])

    frame_transform = np.array(transforms, dtype='float32')
    #cumulative-sum
    path = np.cumsum(frame_transform, axis=0)
    smoothed_path=np.copy(path)
    # shi- Tomasi corner detection
    previous_keypoints = cv2.goodFeaturesToTrack(frame_gray, maxCorners=200, qualityLevel=0.05, minDistance=30.0, blockSize=3 , mask=None, useHarrisDetector=False, k=0.04)
    previous_gray = frame_gray[:]

    return frame_transform

def apply_transformations():
    global frame_queue, frame_queue_indexes, frame_height, frame_width, frame_transforms_smoothed
    queue_frame = frame_queue.popleft()
    queue_frame_index = frame_queue_indexes.popleft()
    bordered_frame = cv2.copyMakeBorder(queue_frame, top=0, bottom=0, left=0, right=0, borderType=cv2.BORDER_CONSTANT, value=[0, 0, 0])
    alpha_bordered_frame = cv2.cvtColor(bordered_frame, cv2.COLOR_BGR2BGRA)
    alpha_bordered_frame[:, :, 3] = 0 
    alpha_bordered_frame[0:0 + frame_height, 0:0 + frame_width, 3] = 255

    dx = frame_transforms_smoothed[queue_frame_index,0]
    dy = frame_transforms_smoothed[queue_frame_index,1]
    da = frame_transforms_smoothed[queue_frame_index,2]

    #2x3 transformation matrix from extracted transformations
    queue_frame_transform = np.zeros((2,3), np.float32)
    queue_frame_transform[0,0] = np.cos(da)
    queue_frame_transform[0,1] = -np.sin(da)
    queue_frame_transform[1,0] = np.sin(da)
    queue_frame_transform[1,1] = np.cos(da)
    queue_frame_transform[0,2] = dx
    queue_frame_transform[1,2] = dy

    frame_wrapped = cv2.warpAffine(alpha_bordered_frame, queue_frame_transform, alpha_bordered_frame.shape[:2][::-1], borderMode=cv2.BORDER_CONSTANT)
    # cv2.imshow("wrap",frame_wrapped)
    #drop alpha channel
    frame_stabilized = frame_wrapped[:, :, :3]
    return frame_stabilized


#Capturing the video using the object video capture 
cap = cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/shaky.mp4")

#Definig queue of length 25 to store the last 25 frames
frame_queue = deque(maxlen=25)
frame_queue_indexes = deque(maxlen=25)


# It is used to improve contrast in images(CLAHE- contrast limited adaptive histogram equalization)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))


smoothed_path = None
path = None
transforms = []
frame_transforms_smoothed = None 
previous_gray = None 
previous_keypoints = None 
frame_height, frame_width = (0, 0)


#Main function 
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        #Resize the frame with width 640 and height 360
        frame = cv2.resize(frame, (640, 360), interpolation = cv2.INTER_LINEAR) 
        # calling the function stabilize(frame)
        stabilized_frame = mask_my_frame(stabilize(frame))
        #Displaying the frames
        cv2.imshow("After", stabilized_frame)
        cv2.imshow("Before", mask_my_frame(frame))
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: break

cap.release()
cv2.destroyAllWindows()